In [1]:
%pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Define Neo4j credentials
NEO4J_URI = "neo4j+s://8e6af5fb.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "6Q7v8t1FcCKs6UkuOeUk83v2GVKqw8xZPvMk6g0hYCg"

# Import necessary libraries
from langchain_community.graphs import Neo4jGraph

# Create Neo4j graph connection
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

# Verify connection
print("Neo4j graph connection established successfully")



Neo4j graph connection established successfully


In [3]:
# Import necessary libraries for document loading and splitting
from langchain_community.document_loaders import WikipediaLoader
from langchain_text_splitters import TokenTextSplitter

# Read the wikipedia article
raw_documents = WikipediaLoader(query="Elizabeth I").load()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
# Split the first 3 documents into smaller chunks for easier processing
documents = text_splitter.split_documents(raw_documents[:3]) 

c:\ProgramData\anaconda3\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\ProgramData\anaconda3\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [4]:
# Import necessary libraries for JSON serialization
import json
import os
from langchain_core.documents import Document

# Define the file path for saving/loading documents
json_file_path = "elizabeth_documents.json"

# Option 1: Save documents to JSON
def save_documents_to_json(docs, file_path):
    """Save LangChain documents to a JSON file."""
    # Convert Document objects to dictionaries
    docs_dict = [{"page_content": doc.page_content, "metadata": doc.metadata} for doc in docs]
    
    # Write to JSON file
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(docs_dict, f, ensure_ascii=False, indent=2)
    
    print(f"Documents saved to {file_path}")

# Option 2: Load documents from JSON
def load_documents_from_json(file_path):
    """Load LangChain documents from a JSON file."""
    if not os.path.exists(file_path):
        print(f"File {file_path} does not exist.")
        return []
    
    with open(file_path, "r", encoding="utf-8") as f:
        docs_dict = json.load(f)
    
    # Convert dictionaries back to Document objects
    docs = [Document(page_content=doc["page_content"], metadata=doc["metadata"]) for doc in docs_dict]
    print(f"Loaded {len(docs)} documents from {file_path}")
    return docs

# Save the current documents to JSON
save_documents_to_json(documents, json_file_path)

# To load documents in the future, uncomment the line below and comment out the Wikipedia loading code
# documents = load_documents_from_json(json_file_path)


Documents saved to elizabeth_documents.json


In [6]:
documents

[Document(page_content='Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last and longest reigning monarch of the House of Tudor. Her eventful reign, and its effect on history and culture, gave name to the Elizabethan era.\nElizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents\' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry\'s death in 1547, Elizabeth\'s younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward\'s will was set aside within weeks of his death and Mar

In [29]:
# Import necessary libraries for LLM and graph transformation
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers.llm import LLMGraphTransformer


llm = ChatOpenAI(model="gpt-4o", temperature=1)
llm_transformer = LLMGraphTransformer(llm)

#extract graph data
graph_documents = llm_transformer.convert_to_graph_documents(documents)

#store to neo4j
graph.add_graph_documents(graph_documents, baseEntityLabel=True, include_source=True)


In [8]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.index.vector.createNodeIndex' has been replaced by 'CREATE VECTOR INDEX')} {position: line: 1, column: 1, offset: 0} for query: 'CALL db.index.vector.createNodeIndex($index_name,$node_label,$embedding_node_property,toInteger($embedding_dimension),$similarity_metric )'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 70, offset: 69} for query: "UNWIND $data AS row MA

In [33]:
# Extract entities from text
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI
from openai_interacter import OpenAIChatInterface


class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

# Define the messages to send to the model
messages = [
    {
        "role": "system",
        "content": "You are extracting organization and person entities from the text."
    },
    {
        "role": "user",
        "content": "Extract the entities from the following text: {question}"
    }
]

# Function to process text and extract entities
def extract_entities(text):
    formatted_messages = [
        {"role": "system", "content": messages[0]["content"]},
        {"role": "user", "content": messages[1]["content"].replace("{question}", text)}
    ]

    print("Input messages: ", formatted_messages)

    # Create a new GPT-4o model instance
    entity_llm = OpenAIChatInterface(model_name="gpt-4o", temperature=1, initial_messages=formatted_messages)
    entity_llm.enable_structured_output(Entities)
    return entity_llm.parse_structured_output()

In [34]:
entities = extract_entities(text = "Where was Amelia Earhart born?")
entities

Input messages:  [{'role': 'system', 'content': 'You are extracting organization and person entities from the text.'}, {'role': 'user', 'content': 'Extract the entities from the following text: Where was Amelia Earhart born?'}]


Entities(names=['Amelia Earhart'])

In [36]:
import re

graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

def remove_lucene_chars(text: str) -> str:
    """
    Remove special characters that have meaning in Lucene syntax.
    
    This function removes or escapes characters that would be interpreted
    as special operators in Lucene full-text search syntax.
    """
    # Remove or escape Lucene special characters
    return re.sub(r'[+\-&|!(){}[\]^"~*?:\\]', ' ', text)

def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines 
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [39]:
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = extract_entities(text =  question)
    print("Entities: ", entities)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "n".join([el['output'] for el in response])
    return result

In [41]:
structured_retriever(question = "Who is Elizabeth I?")

Input messages:  [{'role': 'system', 'content': 'You are extracting organization and person entities from the text.'}, {'role': 'user', 'content': 'Extract the entities from the following text: Who is Elizabeth I?'}]
Entities:  names=['Elizabeth I']


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "


'Elizabeth I - QUEEN_OF -> EnglandnElizabeth I - QUEEN_OF -> IrelandnElizabeth I - AFFILIATION -> House Of TudornElizabeth I - NAMESAKE -> Elizabethan EranElizabeth I - CHILD_OF -> Henry ViiinElizabeth I - CHILD_OF -> Anne BoleynnElizabeth I - SIBLING -> Edward VinElizabeth I - SIBLING -> Mary InElizabeth I - ADVISER -> William CecilnElizabeth I - FOUNDER -> Church Of EnglandnElizabeth I - SUCCEEDED_BY -> James Vi Of ScotlandnElizabeth I - SUPPORTED_BY -> Sir Francis WalsinghamnElizabeth I - FOREIGN_AFFAIRS -> FrancenElizabeth I - FOREIGN_AFFAIRS -> SpainnElizabeth I - NAMED_AFTER -> Elizabethan EranElizabeth I - SUCCESSION -> James Vi Of ScotlandnElizabeth I - DAUGHTER_OF -> Henry ViiinElizabeth I - POSITION_HELD -> Queen Of England And IrelandnElizabeth I - MEMBERSHIP -> House Of TudornElizabeth I - ADVISOR -> William CecilnElizabeth I - RELATED_TO -> Third Succession Act 1543nElizabeth I - RELATED_TO -> Elizabethan Religious SettlementnElizabeth I - GOVERNOR_OF -> English Protestant

In [ ]:
from augmented_llm.llm import AugmentedLLM, LLMProvider
import json

# Create an AugmentedLLM instance with Anthropic provider
llm = AugmentedLLM(
    system_prompt="",
    provider=LLMProvider.ANTHROPIC,
    model_name="claude-3-7-sonnet-20250219",
    temperature=1, 
    max_tokens=64000
)

# Add the structured retriever as a tool
llm.add_tool(
    name="query_knowledge_graph",
    description="Retrieves information about historical entities from a knowledge graph",
    input_schema={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A simple question indicating what information you want to get from the database.", 
                "required": True
            }
        },

    },
    handler=structured_retriever
)

# Test the LLM with a question
response_generator = llm.generate("Tell me about Elizabeth I and her relationships with other historical figures.")

# Print the response
for chunk in response_generator:
    print(chunk, end="", flush=True)


{}